# 自營商選擇權  
* 方法1: 透過openapi取得資訊  
> [三大法人-選擇權買賣權分計-依日期](https://www.taifex.com.tw/cht/3/callsAndPutsDateView)  
* 方法2: 透過期交所網站下載資料進行更新  
> 首頁 > 交易資訊 > 三大法人 > 下載 > 選擇權買賣權分計 > 依日期  
> [選擇權買賣權分計_依日期下載](https://www.taifex.com.tw/cht/3/callsAndPutsDateDown)


In [1]:
import os
from datetime import date
from datetime import datetime
import shutil
import requests
import pandas as pd
import duckdb
import zipfile

In [2]:
# 引用自建公用模組
from proj_util_pkg.settings import ProjEnvSettings

## 公用參數設定

In [3]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [4]:
# 新增偽裝成chrome瀏覽器的標頭
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

## 外部資料讀取  
### 方法1: 透過期交所 open api 取得最近一個交易日數據  

In [ ]:
# 取得三大法人-選擇權買賣權分計-依日期
mj_institutional_trader = requests.get(
    "https://openapi.taifex.com.tw/v1/MarketDataOfMajorInstitutionalTradersDetailsOfCallsAndPutsBytheDate", 
    verify=False,
    headers=headers
).json()
print(mj_institutional_trader)

In [6]:
# json 轉成 dataframe
mj_institutional_trader_df = pd.DataFrame(mj_institutional_trader)
print(mj_institutional_trader_df.shape)
# mj_institutional_trader_df

(30, 16)


In [7]:
# 取得最後一筆交易日，作為方法2的查詢終止日條件
last_txn_date = mj_institutional_trader_df.tail(1)["Date"].values[0]
last_txn_date = pd.to_datetime(last_txn_date)

last_txn_date

Timestamp('2024-10-04 00:00:00')

### 方法2: 透過期交所網站下載資料進行更新  

In [22]:
# 日期參數設定
query_first_date = (last_txn_date - pd.DateOffset(years=3)).strftime("%Y/%m/%d %H:%M")
query_last_date = last_txn_date.strftime("%Y/%m/%d %H:%M")
print(f"{query_first_date} ~ {query_last_date}")

# input_date = "2024/08/05"  # 保留作為手動指定日期
input_date = query_last_date  # 預設指定最近一個交易日期

end_date = pd.to_datetime(input_date)
last_month = (end_date - pd.DateOffset(days=30)).strftime("%Y/%m/%d")

f"{last_month} ~ {input_date}"

2021/10/04 00:00 ~ 2024/10/04 00:00


'2024/09/04 ~ 2024/10/04 00:00'

In [23]:
# 設置請求的URL
url = "https://www.taifex.com.tw/cht/3/callsAndPutsDateDown"

# 設置請求的payload
# payload = {
#     "firstDate": "2021/10/04 00:00",
#     "lastDate": "2024/10/04 00:00",
#     "queryStartDate": "2024/10/04",
#     "queryEndDate": "2024/10/04",
#     "commodityId": "TXO"
# }
payload = {
    "firstDate": query_first_date,
    "lastDate": query_last_date,
    "queryStartDate": last_month,
    "queryEndDate": input_date,
    "commodityId": "TXO"
}
print(payload)

# 發送POST請求
response = requests.post(url, headers=headers, data=payload, verify=False)

# 檢查請求是否成功
if response.status_code == 200:
    # 將回應內容保存為CSV文件
    with open("mj_institutional_trader.csv", "wb") as file:
        file.write(response.content)
    print("CSV文件已成功下載並保存為mj_institutional_trader.csv")

    df = pd.read_csv('mj_institutional_trader.csv', encoding='big5')
    df.loc[:,"日期"] = pd.to_datetime(df["日期"])
    # print(df)

    os.remove("mj_institutional_trader.csv")
else:
    print(f"請求失敗，狀態碼：{response.status_code}")

{'firstDate': '2021/10/04 00:00', 'lastDate': '2024/10/04 00:00', 'queryStartDate': '2024/09/04', 'queryEndDate': '2024/10/04 00:00', 'commodityId': 'TXO'}
CSV文件已成功下載並保存為mj_institutional_trader.csv


/Users/bryson0083/projects/bryson_stock/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.taifex.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [26]:
# 篩選臺指選擇權的自營商資訊
mj_institutional_trader_df = df.copy()

mj_institutional_trader_df = mj_institutional_trader_df[
    (mj_institutional_trader_df["商品名稱"] == "臺指選擇權") & 
    (mj_institutional_trader_df["身份別"] == "自營商")
]
mj_institutional_trader_df = mj_institutional_trader_df[["日期", "未平倉口數買賣淨額"]]

# 依日期對未平倉口數買賣淨額，進行 group by 加總
mj_institutional_trader_df = mj_institutional_trader_df.groupby("日期")["未平倉口數買賣淨額"].sum().reset_index()

mj_institutional_trader_df.columns = ["Date", "net_oi"]
mj_institutional_trader_df

,Date,net_oi
0,2024-09-04,-13701
1,2024-09-05,-6748
2,2024-09-06,-8728
3,2024-09-09,-1162
4,2024-09-10,-3729
5,2024-09-11,-16668
6,2024-09-12,-13185
7,2024-09-13,-12756
8,2024-09-16,-22531
9,2024-09-18,-7109


## 資料留存ＤＢ

In [21]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [27]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [28]:
table_name = "tw_option_proprietary_traders_oi"

In [29]:
# 針對DataFrame，以Date單筆先確認表中，欄位Date沒有重複資料時，才進行單筆insert
for i in range(len(mj_institutional_trader_df)):
    try:
        mj_institutional_trader_df.iloc[i:i+1].to_sql(table_name, conn_duckdb, if_exists="append", index=False)
    except Exception as e:
        # print(e)
        pass


/var/folders/14/b8c6vz6n79n4wt12qny6djk40000gn/T/ipykernel_55428/1829465630.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mj_institutional_trader_df.iloc[i:i+1].to_sql(table_name, conn_duckdb, if_exists="append", index=False)


In [30]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc").fetch_df()

,Date,net_oi
0,2024-10-04,-8718
1,2024-10-01,2587
2,2024-09-30,3179
3,2024-09-27,18104
4,2024-09-26,3999
5,2024-09-25,-4842
6,2024-09-24,-9907
7,2024-09-23,-5153
8,2024-09-20,-1026
9,2024-09-19,5491


In [31]:
# 關閉資料庫連線
conn_duckdb.close()